In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote
import unicodedata
import re
from tqdm.notebook import tqdm
import requests 
import sys
import json
import datetime

In [12]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

In [13]:
with open('password.txt') as f:
    password = f.read()

In [14]:
email = 'nasimbazawadilla'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [15]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)


In [16]:
input_pwd.submit() #enter를 치는 효과
time.sleep(1)

In [17]:
#로그인 정보 저장
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [18]:
#알림설정 - 나중에하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

In [19]:
keyword = '엘피'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%97%98%ED%94%BC'

In [20]:
driver.get(url)
time.sleep(5)

In [21]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [22]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [23]:
#1) 본문 내용 가져오기
try:
    content = soup.select_one('div.C4VMK>span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'📌 크리스마스 이브 및 당일 예약 24일1부 : 17:00 - 19:00 (2h)2부 : 20:00 - 22:00 (2h) - 마감3부 : 22:30 - 00:30 (2h)25일1부 : 17:00 - 19:00 (2h)2부 : 20:00 - 22:00 (2h)3부 : 22:30 - 00:30 (2h)-* 예약시간을 20분 초과할 경우 해당 좌석은 워크인으로 전환됩니다.* 예약문의는 DM 혹은 전화(050713822939)로 가능합니다.'

In [24]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
# #해시태그 정규식
tags = re.findall(r'#[^\s#,\\]+', content)
tags

[]

In [25]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]
date

'2021-12-05'

In [26]:
# 4) 좋아요 수 가져오기
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

'105'

In [27]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place=''
place

'디벨츠잇뮤직'

In [28]:
lis = soup.select_one('a.O4GlU')['href']
lis

'/explore/locations/103879278648133/'

In [29]:
sub_url = insta_url + lis
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [31]:
window._sharedData

vpn = soup.select_one('script.text/javascript')
vpn

NameError: name 'window' is not defined

In [ ]:
row = [content, date, like, place, tags]

In [ ]:
#다음 게시글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
time.sleep(2)

In [ ]:
#다음 게시글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button/div/span').click()
time.sleep(2)

In [ ]:
# 5를 하기위해 팝업 화면을 없앰(x를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

### 5. 반복문으로 데이터 가져오기

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [ ]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [ ]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button/div/span').click()
        time.sleep(1)
    except:
        time.sleep(1)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
        time.sleep(1)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
driver.close()

In [ ]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df

,content,date,like,place,tags
0,📌 크리스마스 이브 및 당일 예약 24일1부 : 17:00 - 19:00 (2h)2...,2021-12-05,105,디벨츠잇뮤직,[]
1,✖️𝑆𝑝𝑖𝑑𝑒𝑟𝑀𝑎𝑛 : 𝐼𝑛𝑡𝑜 𝑡ℎ𝑒 𝑆𝑝𝑖𝑑𝑒𝑟-𝑉𝑒𝑟𝑠𝑒 ✖️...✖️내가 ...,2021-12-05,9seok_lp님 외 146명이 좋아합니,,[]
2,소주가 생각 나는 저녁헬스장 가기싫다,2021-10-26,gmin_vinyl님 외 120명이 좋아합니,,[]
3,“푸른하늘이 무너져버리고 땅이 뒤집힌다 하여도 그런건 중요하지 않아요당신이 날 사랑...,2021-12-06,_n_vinyl님 외 135명이 좋아합니,갤러리아 Gourmet 494,"[#에디트피아프, #샹송, #프랑스, #바이닐, #엘피, #chanson, #엘피바..."
4,🎄⛄️A Charlie Brown Christmas - Vince Guaraldi ...,2021-12-04,gwon_records님 외 110명이 좋아합니,,[]
...,...,...,...,...,...
359,mild high club [going going gone],2021-12-03,a.1님 외 8명이 좋아합니,,[]
360,mild high club [going going gone],2021-12-03,a.1님 외 8명이 좋아합니,,[]
361,mild high club [going going gone],2021-12-03,a.1님 외 8명이 좋아합니,,[]
362,mild high club [going going gone],2021-12-03,a.1님 외 8명이 좋아합니,,[]


In [ ]:
df.drop_duplicates(subset='content', keep='first', inplace=True)
df.shape

(216, 5)

In [ ]:
df.place.value_counts().to_frame()

,place
,231
DOPE RECORDS/Entertainment,76
덕소-팔당구간 한강산책로,6
서울대입구역,5
엘피와음악사랑,4
"Seoul, Korea",3
히피히피,3
사운드카페소리,3
피터판,3
"Queens, New York",2


In [ ]:
df.to_csv('인스타-place.csv', index=False)